In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

In [ ]:
# Load the chains
nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
           [8, 10], [10, 12], [12, 14], [14, 16],
           [16, 18]]

N_steps = 100000000

chain_list = []

for [nb1, nb2] in nb_list:
    chain_name = f'../curve_fit/chains/mcmc_schechter_fit_chain_nb{nb1}-{nb2}.npy'
    flat_samples = np.load(chain_name)[-N_steps:]
    chain_list.append(flat_samples)

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy, levels=[0.683],
                  N_bins=10000):
    if np.any((np.asarray(levels) > 1) | (np.asarray(levels) <= 0)):
        raise Exception('Levels must be in the interval (0, 1]')

    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):
        mask = Z > H_Arr[iii]
        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    out_levels = list()
    for lvl in levels:
        this_H_value = np.interp(lvl, fact_up_Arr, H_Arr)
        out_levels.append(this_H_value)

    return x_centers, y_centers, Z.T, out_levels

In [ ]:
# The parameters are stored as log10(Phistar), log10(Lstar), alpha

bins_Phistar = np.linspace(-8, -5, 50)
bins_Lstar = np.linspace(43, 46, 50)

colors = matplotlib.cm.rainbow(np.linspace(0, 1, len(nb_list)))

fig, ax = plt.subplots(figsize=(6, 4))

for iii, [nb1, nb2] in enumerate(nb_list):
    this_Phistar = chain_list[iii][..., 0]
    this_Lstar = chain_list[iii][..., 1]
    contour_args = contour_curve(this_Phistar, this_Lstar,
                                 bins_Phistar, bins_Lstar,
                                 levels=[0.50])

    ax.contour(*contour_args, colors=[colors[iii]])

plt.show()